<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [19]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [20]:
import folium
import pandas as pd

In [21]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


In [22]:
## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [23]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

# Write the df also locally
spacex_df.to_csv('spacex_launch_geo_local.csv', index=False)

Now, you can take a look at what are the coordinates for each site.


In [39]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [40]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [41]:
#=======================================================================================
#
#   Start Map Creation
#
#=======================================================================================
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
# I fix the cirle color from red to Blue 
#
# circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
#
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='blue', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
#
#
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

#=======================================================================================
#
#   End Map Creation
#
#=======================================================================================

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [42]:
#=======================================================================================
#
#   Start Map Creation - With all sites 
#
#=======================================================================================
# For each launch site, add a Circle and Marker object
# Use the launch_sites_df DataFrame to get launch site coordinates
launch_sites = launch_sites_df.to_dict(orient="records")

# Loop through the launch sites and add Circle and Marker to the map
for site in launch_sites:
    site_name = site["Launch Site"]
    site_lat = site["Lat"]
    site_lon = site["Long"]
    
    # Add a Circle
    circle = folium.Circle(
        location=[site_lat, site_lon],
        radius=1000,  # Adjust the radius as needed
        color='#000000',
        fill=True
    ).add_child(folium.Popup(site_name))
    
    # Add a Marker with a label
    marker = folium.map.Marker(
        [site_lat, site_lon],
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % site_name
        )
    )
    
    # Add the Circle and Marker to the map
    site_map.add_child(circle)
    site_map.add_child(marker)

# Display the map
site_map

#=======================================================================================
#
#   End Map Creation
#
#=======================================================================================


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.

* Proximity to the Equator Line:

It appears that all launch sites are relatively close to the Equator line, which is evident from their positions on the map.
The latitude values for the launch sites are in the range of approximately 28 to 34 degrees North, which places them closer to the Equator compared to higher latitudes.
Proximity to the Coast:

Yes, all launch sites are in relatively close proximity to the coast. They are located near coastal areas or on the coast itself.
The launch sites "CCAFS LC-40," "CCAFS SLC-40," and "KSC LC-39A" are located on the east coast of Florida, near the Atlantic Ocean.
The launch site "VAFB SLC-4E" is located on the west coast of the United States, near the Pacific Ocean.


In [43]:
# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [44]:
spacex_df.tail(20)

,Launch Site,Lat,Long,class
36,KSC LC-39A,28.573255,-80.646895,1
37,KSC LC-39A,28.573255,-80.646895,0
38,KSC LC-39A,28.573255,-80.646895,1
39,KSC LC-39A,28.573255,-80.646895,1
40,KSC LC-39A,28.573255,-80.646895,0
41,KSC LC-39A,28.573255,-80.646895,1
42,KSC LC-39A,28.573255,-80.646895,1
43,KSC LC-39A,28.573255,-80.646895,0
44,KSC LC-39A,28.573255,-80.646895,1
45,KSC LC-39A,28.573255,-80.646895,1


In [59]:
# Create a MarkerCluster object
marker_cluster = MarkerCluster()

# Loop through the launch records and add markers to the marker cluster
for index, row in spacex_df.iterrows():
    launch_site = row['Launch Site']
    site_lat = row['Lat']
    site_lon = row['Long']
    launch_class = row['class']
    
    # Determine the marker color based on launch outcome
    marker_color = 'green' if launch_class == 1 else 'red'
    
    # Create a Marker with a color based on launch outcome
    marker = folium.map.Marker(
        [site_lat, site_lon],
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12; color:{marker_color};"><b>{launch_site}</b></div>'
        )
    )
    
    # Add the marker to the MarkerCluster
    marker_cluster.add_child(marker)

# Add the MarkerCluster to the site_map
site_map.add_child(marker_cluster)

# Display the map
site_map

In [60]:
#===========================================================================
# Create a new column 'marker_color' based on the 'class' value in spacex_df
#===========================================================================


spacex_df['marker_color'] = spacex_df.apply(lambda row: 'green' if row['class'] == 1 else 'red', axis=1)

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red


In [61]:
spacex_df.head(20)

,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red
5,CCAFS LC-40,28.562302,-80.577356,0,red
6,CCAFS LC-40,28.562302,-80.577356,0,red
7,CCAFS LC-40,28.562302,-80.577356,0,red
8,CCAFS LC-40,28.562302,-80.577356,0,red
9,CCAFS LC-40,28.562302,-80.577356,0,red


In [62]:
#========================================================================
#
# Main code Todo 2
#
#========================================================================
# Create another Folium map
#==============================================================
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
#==============================================================

#==============================================================
# Create a MarkerCluster object to group markers
marker_cluster = MarkerCluster()
#==============================================================

#==============================================================
# Add the MarkerCluster to the site_map
site_map.add_child(marker_cluster)
#==============================================================
# For each launch site, add a Circle and Marker object
for index, row in launch_sites_df.iterrows():
    site_coordinate = [row['Lat'], row['Long']]
    site_name = row['Launch Site']
    
    # Create a circle at the site's coordinate with a popup label showing its name
    circle = folium.Circle(site_coordinate, radius=1000, color='blue', fill=True).add_child(folium.Popup(site_name))
    
    # Create a marker with an icon as a text label
    marker = folium.map.Marker(
        site_coordinate,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % site_name,
        )
    )
    
    # Add the circle and marker to the map
    site_map.add_child(circle)
    site_map.add_child(marker)

# Iterate through each row in the spacex_df DataFrame
for index, record in spacex_df.iterrows():
    # Create a Marker object for each launch record with the updated marker_color
    marker = folium.Marker(
        [record['Lat'], record['Long']],
        icon=folium.Icon(color='white', icon_color=record['marker_color'])
    )
    # Add the created marker to the marker_cluster
    marker_cluster.add_child(marker)

# Display the map
site_map

#========================================================================
#
# Main code
#
#========================================================================

In [74]:
#========================================================================
#
# Main code Todo 2 Version 4
#
#========================================================================
# Create another Folium map
#==============================================================

site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
#==============================================================

#==============================================================
# Create a MarkerCluster object to group markers
marker_cluster = MarkerCluster()
#========================================================================
# Add green markers for the coastline coordinates
#========================================================================
# Functions 
#========================================================================
import math

def calculate_distance(lat1, lon1, lat2, lon2):
    # Calculate the distance between two coordinates using the Haversine formula
    R = 6371  # Radius of the Earth in kilometers
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c
    return distance








#========================================================================
# Functions 
#========================================================================


# Define the fixed coastline coordinates for the specified launch sites

coastline_coordinates = {
    "KSC LC-39A": (28.57712, -80.57163),
    "CCAFS LC-40": (28.56289, -80.56772),
    "CCAFS SLC-40": (28.56342, -80.56802),
    "VAFB SLC-4E": (34.63572, -120.62541),  # Corrected coordinate for VAFB SLC-4E
}

#==============================================================
for site_name, coordinates in coastline_coordinates.items():
    site_marker = folium.Marker(
        location=coordinates,
        icon=folium.Icon(color='green')
    )
    site_map.add_child(site_marker)

# Create PolyLines from each launch site to the selected coastline point
for index, row in launch_sites_df.iterrows():
    site_coordinate = [row['Lat'], row['Long']]
    site_name = row['Launch Site']
    
    coastline_lat, coastline_lon = coastline_coordinates.get(site_name, (0, 0))
    
    if coastline_lat != 0 and coastline_lon != 0:
        # Create a PolyLine from the launch site to the coastline point
        lines = folium.PolyLine(
            locations=[site_coordinate, (coastline_lat, coastline_lon)],
            color='blue',
            weight=1
        )
        site_map.add_child(lines)
#========================================================================
# Add green markers for the coastline coordinates
#========================================================================

#========================================================================
# Add red markers for the railway coordinates
#========================================================================

railway_coordinates = {
    "KSC LC-39A": (28.60303, -80.59091),
    "CCAFS LC-40": (28.56354, -80.58686),
    "CCAFS SLC-40": (28.56354, -80.58686),
    "VAFB SLC-4E": (34.63765, -120.6234),  # Corrected coordinate for VAFB SLC-4E
}

# Create red markers for the Railway coordinates
for site_name, site_coordinate in railway_coordinates.items():
    folium.Marker(
        location=site_coordinate,
        icon=folium.Icon(color='red'),
        tooltip=site_name,
    ).add_to(site_map)

    # Create PolyLines from each launch site to the railway
for index, row in launch_sites_df.iterrows():
    site_coordinate = [row['Lat'], row['Long']]
    site_name = row['Launch Site']
    
    railway_lat, railway_lon = railway_coordinates.get(site_name, (0, 0))
    
    if railway_lat != 0 and railway_lon != 0:
        # Create a PolyLine from the launch site to the railway point
        lines = folium.PolyLine(
            locations=[site_coordinate, (railway_lat, railway_lon)],
            color='red',
            weight=1,
            opacity=0.6
        )
        site_map.add_child(lines)    
#========================================================================
# Add red markers for the railway coordinates
#========================================================================

#========================================================================
# Add yellow markers for the Highway coordinates
#========================================================================    
# Define the highway coordinates
highway_coordinates = {
    "KSC LC-39A": (28.58669, -80.65707),
    "CCAFS LC-40": (28.58669, -80.65707),
    "CCAFS SLC-40": (28.58669, -80.65707),
    "VAFB SLC-4E": (34.64189, -120.61855),  # Corrected coordinate for VAFB SLC-4E
}

# Create orange markers for the Highway coordinates
for site_name, site_coordinate in highway_coordinates.items():
    folium.Marker(
        location=site_coordinate,
        icon=folium.Icon(color='orange'),  # Use 'orange' instead of 'yellow'
        tooltip=site_name,
    ).add_to(site_map)  
    
# Create PolyLines from each launch site to the selected highway point
for index, row in launch_sites_df.iterrows():
    site_coordinate = [row['Lat'], row['Long']]
    site_name = row['Launch Site']
    
    highway_lat, highway_lon = highway_coordinates.get(site_name, (0, 0))
    
    if highway_lat != 0 and highway_lon != 0:
        # Create a PolyLine from the launch site to the highway point
        lines = folium.PolyLine(
            locations=[site_coordinate, (highway_lat, highway_lon)],
            color='orange',  # Use 'orange' to match the marker color
            weight=1
        )
        site_map.add_child(lines)
#========================================================================
# Add blue markers for the City coordinates
#========================================================================
city_coordinates = {
    "KSC LC-39A": (28.53672, -81.38165),
    "CCAFS LC-40": (28.53672, -81.38165),
    "CCAFS SLC-40": (28.53672, -81.38165),
    "VAFB SLC-4E": (34.69843, -120.45784),  # Corrected coordinate for VAFB SLC-4E
}
                
    
# Create blue markers for the City coordinates
for site_name, site_coordinate in city_coordinates.items():
    folium.Marker(
        location=site_coordinate,
        icon=folium.Icon(color='blue'),  # Set the marker color to 'blue'
        tooltip=site_name,
    ).add_to(site_map)    

    # Create PolyLines from each launch site to the selected city point
for index, row in launch_sites_df.iterrows():
    site_coordinate = [row['Lat'], row['Long']]
    site_name = row['Launch Site']
    
    city_lat, city_lon = city_coordinates.get(site_name, (0, 0))
    
    if city_lat != 0 and city_lon != 0:
        # Create a PolyLine from the launch site to the city point
        lines = folium.PolyLine(
            locations=[site_coordinate, (city_lat, city_lon)],
            color='blue',  # Use 'blue' to match the marker color
            weight=1
        )
        site_map.add_child(lines)
    
    
#========================================================================
# Add blue markers for the City coordinates
#========================================================================




#==============================================================
# Add the MarkerCluster to the site_map
site_map.add_child(marker_cluster)
#==============================================================
# Create a circle at the site's coordinate with a popup label showing its name, success, failure, distance to coastline,
# total launches, and distance to the nearest launch site

# Calculate the total launches for each launch site using the DataFrame
total_launches = spacex_df['Launch Site'].value_counts().to_dict()


for index, row in launch_sites_df.iterrows():
    site_coordinate = [row['Lat'], row['Long']]
    site_name = row['Launch Site']

    # Calculate the success and failure percentages for the current site
    site_success_percentage = success_percentage.get(site_name, 0)  # Get the site's success percentage
    site_failure_percentage = failure_percentage.get(site_name, 0)  # Get the site's failure percentage

    # Calculate the distance to the fixed coastline point
    coastline_lat, coastline_lon = coastline_coordinates.get(site_name, (0, 0))
    distance_to_coastline = calculate_distance(row['Lat'], row['Long'], coastline_lat, coastline_lon)

    # Calculate the nearest launch site and its distance
    nearest_site_name = None
    nearest_distance = float('inf')  # Initialize with a large value
    for other_site_name, other_row in launch_sites_df.iterrows():
        if site_name != other_row['Launch Site']:
            other_coordinate = [other_row['Lat'], other_row['Long']]
            distance = calculate_distance(row['Lat'], row['Long'], other_coordinate[0], other_coordinate[1])
            if distance < nearest_distance:
                nearest_distance = distance
                nearest_site_name = other_row['Launch Site']

    # Calculate the distance to railway, highway, and city coordinates
    railway_lat, railway_lon = railway_coordinates.get(site_name, (0, 0))
    highway_lat, highway_lon = highway_coordinates.get(site_name, (0, 0))
    city_lat, city_lon = city_coordinates.get(site_name, (0, 0))

    # Calculate distances using the Haversine formula
    distance_to_railway = calculate_distance(row['Lat'], row['Long'], railway_lat, railway_lon)
    distance_to_highway = calculate_distance(row['Lat'], row['Long'], highway_lat, highway_lon)
    distance_to_city = calculate_distance(row['Lat'], row['Long'], city_lat, city_lon)
    
    
    # Create a circle at the site's coordinate with a popup label showing its name, success, failure, distance to coastline,
    # total launches, and distance to the nearest launch site
   
    # Now you can use total_launches in your popup content
    popup_content = f"<b>{site_name}</b><br>-----------------------------------------------------------<br>Total Launches - {total_launches.get(site_name, 0)}<br>Success - {site_success_percentage:.2f}%<br>Failure - {site_failure_percentage:.2f}%<br>Distance to Coastline - {distance_to_coastline:.2f} KM<br>Distance to Railway - {distance_to_railway:.2f} KM<br>Distance to Highway - {distance_to_highway:.2f} KM<br>Distance to City - {distance_to_city:.2f} KM<br>Distance to Nearest Site - {nearest_site_name} - {nearest_distance:.2f} KM<br>-----------------------------------------------------------"
    circle = folium.Circle(
        site_coordinate, 
        radius=1500,  # Increase the radius to accommodate longer text
        color='black',  # Set the border color to black
        fill_color='yellow',  # Set the fill color to yellow
        fill=True
    ).add_child(
        folium.Popup(popup_content, max_width=400)  # Increase the max_width of the popup
    )

    # Create a variable with the combined site name and success percentage
    site_label = f"{site_name}"

    # Create a text label to display under the circle with the same color as the circle
    text_label = folium.DivIcon(
        icon_size=(50, 20),  # Increased icon size for larger text
        icon_anchor=(0, 0),
        html=f'<div style="font-size: 14; color:black;"><b>{site_name}</b></div>'  # Set the font color to black
    )

    # Create a marker with an icon as a text label for launch sites, red text
    marker = folium.map.Marker(
        site_coordinate,
        icon=DivIcon(
            icon_size=(50, 20),  # Increased icon size for larger text
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 14; color:red;"><b>{site_name}</b></div>'  # Increased font size
        )
    )

    # Add the circle, text label, and marker to the map and marker_cluster
    # Add the modified circle and text label to the map and marker_cluster
    site_map.add_child(circle)
    site_map.add_child(folium.Marker(location=site_coordinate, icon=text_label))  # Add the text label under the circle
    marker_cluster.add_child(marker)    
    
# Iterate through each row in the spacex_df DataFrame
for index, record in spacex_df.iterrows():
    # Create a Marker object for each launch record with the updated marker_color
    marker = folium.Marker(
        [record['Lat'], record['Long']],
        icon=folium.Icon(color='white', icon_color=record['marker_color'])
    )
    # Add the created marker to the marker_cluster
    marker_cluster.add_child(marker)


#========================================================================
#
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
#
#========================================================================
    

formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)

# Display the map
site_map


#========================================================================
#
# Main code
#
#========================================================================



In [75]:
#========================================================================
#
# Main code Todo 2 Version 5 - FINAL 
#
#========================================================================
# Create another Folium map
#==============================================================

site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
#==============================================================

#==============================================================
# Create a MarkerCluster object to group markers
marker_cluster = MarkerCluster()
#========================================================================
# Add green markers for the coastline coordinates
#========================================================================
# Functions 
#========================================================================
import math

def calculate_distance(lat1, lon1, lat2, lon2):
    # Calculate the distance between two coordinates using the Haversine formula
    R = 6371  # Radius of the Earth in kilometers
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c
    return distance

#========================================================================
# Functions 
#========================================================================


# Define the fixed coastline coordinates for the specified launch sites
coastline_coordinates = {
    "KSC LC-39A": (28.57712, -80.57163),
    "CCAFS LC-40": (28.56289, -80.56772),
    "CCAFS SLC-40": (28.56342, -80.56802),
    "VAFB SLC-4E": (34.63572, -120.62541),  # Corrected coordinate for VAFB SLC-4E
}

# Marker
for site_name, coordinates in coastline_coordinates.items():
    site_marker = folium.Marker(
        location=coordinates,
        icon=folium.Icon(color='green')
    )

    # Calculate the distance from the launch site to the coastline point
    for index, row in launch_sites_df.iterrows():
        if row['Launch Site'] == site_name:
            distance_to_coastline = calculate_distance(row['Lat'], row['Long'], coordinates[0], coordinates[1])
            break
    else:
        distance_to_coastline = 0  # Default to 0 if no matching launch site is found

    # Add the distance to the coastline to the marker's label with a wider max_width
    site_marker.add_child(folium.Popup(f"Distance to Coastline: {distance_to_coastline:.2f} KM", max_width=300))
    
    site_map.add_child(site_marker)

# Create PolyLines from each launch site to the selected coastline point
for index, row in launch_sites_df.iterrows():
    site_coordinate = [row['Lat'], row['Long']]
    site_name = row['Launch Site']
    
    coastline_lat, coastline_lon = coastline_coordinates.get(site_name, (0, 0))
    
    if coastline_lat != 0 and coastline_lon != 0:
        # Create a PolyLine from the launch site to the coastline point
        lines = folium.PolyLine(
            locations=[site_coordinate, (coastline_lat, coastline_lon)],
            color='blue',
            weight=1
        )
        site_map.add_child(lines)
#========================================================================
# Add green markers for the coastline coordinates
#========================================================================

#========================================================================
# Add red markers for the railway coordinates
#========================================================================

railway_coordinates = {
    "KSC LC-39A": (28.60303, -80.59091),
    "CCAFS LC-40": (28.56354, -80.58686),
    "CCAFS SLC-40": (28.56354, -80.58686),
    "VAFB SLC-4E": (34.63765, -120.6234),  # Corrected coordinate for VAFB SLC-4E
}

# Create red markers for the Railway coordinates and add distance labels
for site_name, site_coordinate in railway_coordinates.items():
    site_marker = folium.Marker(
        location=site_coordinate,
        icon=folium.Icon(color='red'),
        tooltip=site_name,
    )

    # Calculate the distance from the launch site to the railway point
    for index, row in launch_sites_df.iterrows():
        if row['Launch Site'] == site_name:
            distance_to_railway = calculate_distance(row['Lat'], row['Long'], site_coordinate[0], site_coordinate[1])
            break
    else:
        distance_to_railway = 0  # Default to 0 if no matching launch site is found

    # Add the distance to the railway to the marker's label with a wider max_width
    site_marker.add_child(folium.Popup(f"Distance to Railway: {distance_to_railway:.2f} KM", max_width=300))
    
    site_marker.add_to(site_map)

    # Create PolyLines from each launch site to the railway
    for index, row in launch_sites_df.iterrows():
        site_coordinate = [row['Lat'], row['Long']]
        site_name = row['Launch Site']

        railway_lat, railway_lon = railway_coordinates.get(site_name, (0, 0))

        if railway_lat != 0 and railway_lon != 0:
            # Create a PolyLine from the launch site to the railway point
            lines = folium.PolyLine(
                locations=[site_coordinate, (railway_lat, railway_lon)],
                color='red',
                weight=1,
                opacity=0.6
            )
            site_map.add_child(lines)  
#========================================================================
# Add red markers for the railway coordinates
#========================================================================

#========================================================================
# Add yellow markers for the Highway coordinates
#========================================================================    
# Define the highway coordinates
highway_coordinates = {
    "KSC LC-39A": (28.58669, -80.65707),
    "CCAFS LC-40": (28.58669, -80.65707),
    "CCAFS SLC-40": (28.58669, -80.65707),
    "VAFB SLC-4E": (34.64189, -120.61855),  # Corrected coordinate for VAFB SLC-4E
}

# Create orange markers for the Highway coordinates and add distance labels
for site_name, site_coordinate in highway_coordinates.items():
    site_marker = folium.Marker(
        location=site_coordinate,
        icon=folium.Icon(color='orange'),  # Use 'orange' instead of 'yellow'
        tooltip=site_name,
    )

    # Calculate the distance from the launch site to the highway point
    for index, row in launch_sites_df.iterrows():
        if row['Launch Site'] == site_name:
            distance_to_highway = calculate_distance(row['Lat'], row['Long'], site_coordinate[0], site_coordinate[1])
            break
    else:
        distance_to_highway = 0  # Default to 0 if no matching launch site is found

    # Add the distance to the highway to the marker's label with a wider max_width
    site_marker.add_child(folium.Popup(f"Distance to Highway: {distance_to_highway:.2f} KM", max_width=300))
    
    site_marker.add_to(site_map)

    # Create PolyLines from each launch site to the highway
    for index, row in launch_sites_df.iterrows():
        site_coordinate = [row['Lat'], row['Long']]
        site_name = row['Launch Site']

        highway_lat, highway_lon = highway_coordinates.get(site_name, (0, 0))

        if highway_lat != 0 and highway_lon != 0:
            # Create a PolyLine from the launch site to the highway point
            lines = folium.PolyLine(
                locations=[site_coordinate, (highway_lat, highway_lon)],
                color='orange',  # Use 'orange' to match the marker color
                weight=1
            )
            site_map.add_child(lines)

#========================================================================
# Add blue markers for the City coordinates
#========================================================================
city_coordinates = {
    "KSC LC-39A": (28.53672, -81.38165),
    "CCAFS LC-40": (28.53672, -81.38165),
    "CCAFS SLC-40": (28.53672, -81.38165),
    "VAFB SLC-4E": (34.69843, -120.45784),  # Corrected coordinate for VAFB SLC-4E
}

# Create blue markers for the City coordinates and add distance labels
for site_name, site_coordinate in city_coordinates.items():
    site_marker = folium.Marker(
        location=site_coordinate,
        icon=folium.Icon(color='blue'),  # Set the marker color to 'blue'
        tooltip=site_name,
    )

    # Calculate the distance from the launch site to the city point
    for index, row in launch_sites_df.iterrows():
        if row['Launch Site'] == site_name:
            distance_to_city = calculate_distance(row['Lat'], row['Long'], site_coordinate[0], site_coordinate[1])
            break
    else:
        distance_to_city = 0  # Default to 0 if no matching launch site is found

    # Add the distance to the city to the marker's label with a wider max_width
    site_marker.add_child(folium.Popup(f"Distance to City: {distance_to_city:.2f} KM", max_width=300))
    
    site_marker.add_to(site_map)

    # Create PolyLines from each launch site to the city
    for index, row in launch_sites_df.iterrows():
        site_coordinate = [row['Lat'], row['Long']]
        site_name = row['Launch Site']

        city_lat, city_lon = city_coordinates.get(site_name, (0, 0))

        if city_lat != 0 and city_lon != 0:
            # Create a PolyLine from the launch site to the city point
            lines = folium.PolyLine(
                locations=[site_coordinate, (city_lat, city_lon)],
                color='blue',  # Use 'blue' to match the marker color
                weight=1
            )
            site_map.add_child(lines)

    
    
#========================================================================
# Add blue markers for the City coordinates
#========================================================================




#==============================================================
# Add the MarkerCluster to the site_map
site_map.add_child(marker_cluster)
#==============================================================
# Create a circle at the site's coordinate with a popup label showing its name, success, failure, distance to coastline,
# total launches, and distance to the nearest launch site

# Calculate the total launches for each launch site using the DataFrame
total_launches = spacex_df['Launch Site'].value_counts().to_dict()


for index, row in launch_sites_df.iterrows():
    site_coordinate = [row['Lat'], row['Long']]
    site_name = row['Launch Site']

    # Calculate the success and failure percentages for the current site
    site_success_percentage = success_percentage.get(site_name, 0)  # Get the site's success percentage
    site_failure_percentage = failure_percentage.get(site_name, 0)  # Get the site's failure percentage

    # Calculate the distance to the fixed coastline point
    coastline_lat, coastline_lon = coastline_coordinates.get(site_name, (0, 0))
    distance_to_coastline = calculate_distance(row['Lat'], row['Long'], coastline_lat, coastline_lon)

    # Calculate the nearest launch site and its distance
    nearest_site_name = None
    nearest_distance = float('inf')  # Initialize with a large value
    for other_site_name, other_row in launch_sites_df.iterrows():
        if site_name != other_row['Launch Site']:
            other_coordinate = [other_row['Lat'], other_row['Long']]
            distance = calculate_distance(row['Lat'], row['Long'], other_coordinate[0], other_coordinate[1])
            if distance < nearest_distance:
                nearest_distance = distance
                nearest_site_name = other_row['Launch Site']

    # Calculate the distance to railway, highway, and city coordinates
    railway_lat, railway_lon = railway_coordinates.get(site_name, (0, 0))
    highway_lat, highway_lon = highway_coordinates.get(site_name, (0, 0))
    city_lat, city_lon = city_coordinates.get(site_name, (0, 0))

    # Calculate distances using the Haversine formula
    distance_to_railway = calculate_distance(row['Lat'], row['Long'], railway_lat, railway_lon)
    distance_to_highway = calculate_distance(row['Lat'], row['Long'], highway_lat, highway_lon)
    distance_to_city = calculate_distance(row['Lat'], row['Long'], city_lat, city_lon)
    
    
    # Create a circle at the site's coordinate with a popup label showing its name, success, failure, distance to coastline,
    # total launches, and distance to the nearest launch site
   
    # Now you can use total_launches in your popup content
    popup_content = f"<b>{site_name}</b><br>-----------------------------------------------------------<br>Total Launches - {total_launches.get(site_name, 0)}<br>Success - {site_success_percentage:.2f}%<br>Failure - {site_failure_percentage:.2f}%<br>Distance to Coastline - {distance_to_coastline:.2f} KM<br>Distance to Railway - {distance_to_railway:.2f} KM<br>Distance to Highway - {distance_to_highway:.2f} KM<br>Distance to City - {distance_to_city:.2f} KM<br>Distance to Nearest Site - {nearest_site_name} - {nearest_distance:.2f} KM<br>-----------------------------------------------------------"
    circle = folium.Circle(
        site_coordinate, 
        radius=1500,  # Increase the radius to accommodate longer text
        color='black',  # Set the border color to black
        fill_color='yellow',  # Set the fill color to yellow
        fill=True
    ).add_child(
        folium.Popup(popup_content, max_width=400)  # Increase the max_width of the popup
    )

    # Create a variable with the combined site name and success percentage
    site_label = f"{site_name}"

    # Create a text label to display under the circle with the same color as the circle
    text_label = folium.DivIcon(
        icon_size=(50, 20),  # Increased icon size for larger text
        icon_anchor=(0, 0),
        html=f'<div style="font-size: 14; color:black;"><b>{site_name}</b></div>'  # Set the font color to black
    )

    # Create a marker with an icon as a text label for launch sites, red text
    marker = folium.map.Marker(
        site_coordinate,
        icon=DivIcon(
            icon_size=(50, 20),  # Increased icon size for larger text
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 14; color:red;"><b>{site_name}</b></div>'  # Increased font size
        )
    )

    # Add the circle, text label, and marker to the map and marker_cluster
    # Add the modified circle and text label to the map and marker_cluster
    site_map.add_child(circle)
    site_map.add_child(folium.Marker(location=site_coordinate, icon=text_label))  # Add the text label under the circle
    marker_cluster.add_child(marker)    
    
# Iterate through each row in the spacex_df DataFrame
for index, record in spacex_df.iterrows():
    # Create a Marker object for each launch record with the updated marker_color
    marker = folium.Marker(
        [record['Lat'], record['Long']],
        icon=folium.Icon(color='white', icon_color=record['marker_color'])
    )
    # Add the created marker to the marker_cluster
    marker_cluster.add_child(marker)


#========================================================================
#
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
#
#========================================================================
    

formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)

# Display the map
site_map


#========================================================================
#
# Main code Todo 2 Version 5 - FINAL 
#
#========================================================================

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


Proximity to Railways:

SpaceX Launch Sites: KSC LC-39A, CCAFS LC-40, CCAFS SLC-40, VAFB SLC-4E
Railways are crucial for transporting heavy payloads and equipment. 
Proximity to railways can simplify logistics and reduce transportation costs.
By plotting distance lines to the nearest railway points, we can determine if SpaceX launch sites are strategically located near railway infrastructure.
Proximity to Highways:

SpaceX Launch Sites: KSC LC-39A, CCAFS LC-40, CCAFS SLC-40, VAFB SLC-4E
Highways provide easy access for the transportation of equipment and personnel to launch sites.
Mapping distance lines to the nearest highway points allows us to assess whether SpaceX's launch sites are conveniently connected to major road networks.
Proximity to Coastlines:

SpaceX Launch Sites: KSC LC-39A, CCAFS LC-40, CCAFS SLC-40, VAFB SLC-4E
Coastlines offer launch sites the advantage of launching over the open ocean, 
reducing risks to populated areas in case of mission anomalies.
Examining the distances from launch sites to coastlines will reveal 
their strategic positioning for safe and efficient launches.
Distance from Cities:

SpaceX Launch Sites: KSC LC-39A, CCAFS LC-40, CCAFS SLC-40, VAFB SLC-4E
Launch sites need to maintain a certain distance from populated areas 
to ensure public safety.
We will analyze the distances between launch sites and nearby cities, 
providing insights into SpaceX's commitment to safety and environmental 
considerations.Proximity to Railways:

SpaceX Launch Sites: KSC LC-39A, CCAFS LC-40, CCAFS SLC-40, VAFB SLC-4E
Railways are crucial for transporting heavy payloads and equipment. 
Proximity to railways can simplify logistics and reduce transportation costs.
By plotting distance lines to the nearest railway points, we can determine 
if SpaceX launch sites are strategically located near railway infrastructure.
Proximity to Highways:

SpaceX Launch Sites: KSC LC-39A, CCAFS LC-40, CCAFS SLC-40, VAFB SLC-4E
Highways provide easy access for the transportation of equipment and 
personnel to launch sites.
Mapping distance lines to the nearest highway points allows us to assess 
whether SpaceX's launch sites are conveniently connected to major road networks.
Proximity to Coastlines:

SpaceX Launch Sites: KSC LC-39A, CCAFS LC-40, CCAFS SLC-40, VAFB SLC-4E
Coastlines offer launch sites the advantage of launching over the open ocean, 
reducing risks to populated areas in case of mission anomalies.
Examining the distances from launch sites to coastlines will reveal 
their strategic positioning for safe and efficient launches.


Distance from Cities:

SpaceX Launch Sites: KSC LC-39A, CCAFS LC-40, CCAFS SLC-40, VAFB SLC-4E
Launch sites need to maintain a certain distance from populated areas to ensure public safety.
We will analyze the distances between launch sites and nearby cities, providing insights into SpaceX's commitment to safety and environmental considerations.

# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2022-01-01)


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
